# Overview Materi

Source: https://www.youtube.com/watch?v=LDRbO9a6XPU

Jelaskan secara singkat apa itu decision tree menurut pemahamanmu!

Decision Tree adalah metode analisis atau algoritma yang digunakan untuk mengambil keputusan dengan memecah suatu permasalahan ke dalam bentuk struktur pohon, bercabang. Strukturnya dimulai dengan mempertanyakan suatu kondisi tertentu, memberikan beberapa pilihan atas kondisi tersebut, lalu yang terakhir ada hasil keputusan atau outputnya. Secara garis besar, decision tree membantu menggambarkan proses pengambilan keputusan secara sistematis dan mudah dipahami, sebab mengikuti alur if-then (jika-maka) sampai menghasilkan keputusan akhir yang sesuai dengan kondisi tertentu yang terjadi.

# Import Data & Libraries

In [9]:
from __future__ import print_function

# label kolom
header = ["color", "diameter", "label"]

# data training
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# data testing
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],]

# Fungsi Dasar

In [11]:
# fungsi mencari apa saja unique value dari suatu kolom
def unique_vals(rows, col):
    return set([row[col] for row in rows])

# contoh penggunaan
print(unique_vals(training_data, 0))
print(unique_vals(training_data, 1))
print(unique_vals(training_data, 2))

{'Green', 'Red', 'Yellow'}
{1, 3}
{'Apple', 'Grape', 'Lemon'}


In [12]:
# fungsi Menghitung jumlah unique value dari suatu kolom
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# contoh penggunaan
print(class_counts(training_data))

{'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [13]:
# fungsi pengecekan suatu value numerik atau bukan
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

# contoh penggunaan
print(is_numeric(7))
print(is_numeric("Red"))
print(is_numeric(7.5))

True
False
True


In [14]:
# kelas untuk merepresentasikan pertanyaan pada decision tree
class Question:

    # inisialisasi kolom dan nilai pertanyaan
    def __init__(self, column, value):
        self.column = column
        self.value = value

    # fungsi untuk mengecek apakah suatu contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # fungsi untuk menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

    # mengecek apakah contoh data sesuai dengan pertanyaan
    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    # menampilkan pertanyaan dalam format string yang mudah dibaca
    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

# contoh penggunaan 1
print(Question(1, 3))

# contoh penggunaan 2
q = Question(0, 'Green')
print(q)

# contoh penggunaan 3
example = training_data[0]
print(q.match(example))

Is diameter >= 3?
Is color == Green?
True


In [15]:
# membagi dataset menjadi dua berdasarkan pertanyaan
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# contoh penggunaan
q = Question(0, 'Green')
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
print(true_rows)
print(false_rows)

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]
[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]


**apa itu gini impurity?**
<br> gini impurity berfungsi mengukur tingkat ketidakmurnian atau ketidakteraturan pada sebuah simpul (node) dalam pohon

In [16]:
# menghitung nilai Gini Impurity untuk sebuah dataset
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
        return impurity

# contoh penggunaan
print("Gini seluruh data training:", gini(training_data))

Gini seluruh data training: 0.84


**apa itu information gain?**
<br> information gain berfungsi mengukur seberapa efektif sebuah fitur dalam memisahkan data berdasarkan kelas-kelasnya

In [17]:
# menghitung nilai Information Gain dari pemisahan dataset
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

# contoh penggunaan
q = Question(0, 'Green')

true_rows, false_rows = partition(training_data, q)
current_uncertainty = gini(training_data)
gain = info_gain(true_rows, false_rows, current_uncertainty)

print("Pertanyaan:", q)
print("True rows:", true_rows)
print("False rows:", false_rows)
print("Current uncertainty:", current_uncertainty)
print("Gain:", gain)

Pertanyaan: Is color == Green?
True rows: [['Green', 3, 'Apple']]
False rows: [['Yellow', 3, 'Apple'], ['Red', 1, 'Grape'], ['Red', 1, 'Grape'], ['Yellow', 3, 'Lemon']]
Current uncertainty: 0.84
Gain: 0.08999999999999997


In [18]:
# mencari pertanyaan terbaik untuk membagi dataset berdasarkan information gain tertinggi
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1
    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the dataset
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

# contoh penggunaan
best_gain, best_question = find_best_split(training_data)
print("Best gain:", best_gain)
print("Best question:", best_question)

Best gain: 0.5066666666666666
Best question: Is diameter >= 3?


# Fungsi Decision Tree

In [19]:
# merepresentasikan node daun (leaf) pada decision tree yang berisi hasil prediksi
class Leaf:

    # inisialisasi leaf dengan menghitung jumlah kemunculan tiap kelas
    def __init__(self, rows):
        self.predictions = class_counts(rows)

leaf = Leaf(training_data)
print("Prediksi di leaf:", leaf.predictions)


Prediksi di leaf: {'Apple': 2, 'Grape': 2, 'Lemon': 1}


In [20]:
# merepresentasikan node keputusan (decision node) yang berisi pertanyaan dan cabang
class Decision_Node:

    # inisialisasi node dengan pertanyaan, cabang benar, dan cabang salah
    def __init__(self, question, true_branch, false_branch):
       self.question = question,
       self.true_brancah = true_branch,
       self.false_branch = false_branch


In [21]:
# membangun decision tree secara rekursif
def build_tree(rows):
  gain, question = find_best_split(rows)

  true_rows, false_rows = partition(rows, question)
  true_branch = build_tree(true_rows)
  false_branch = build_tree(false_rows)

  return Decision_Node(question, true_branch, false_branch)

In [22]:
# mencetak struktur decision tree secara rekursif dalam format teks
def print_tree(node, spacing=""):


    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return

    # mencetak pertanyaan pada node saat ini
    print(spacing + str(node.question))

    # mencetak cabang true secara rekursif
    print(spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

    # mencetak cabang false secara rekursif
    print(spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [29]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):

    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
      return classify(row, node.false_branch)

In [28]:
# mengklasifikasikan satu baris data menggunakan decision tree
def classify(row, node):


    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

    # base case: jika sudah mencapai leaf
    if isinstance(node, Leaf):
        return node.predictions

    # menentukan apakah mengikuti cabang true atau cabang false
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

    # dengan membandingkan nilai fitur pada baris dengan pertanyaan di node
    my_tree = build_tree(training_data)
    example = testing_data[1]


In [25]:
# menampilkan prediksi pada leaf dalam format persentase
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        return probs

# Predict Using Decision Tree

In [27]:
# menguji decision tree dengan data uji dan membandingkan hasil prediksi dengan label asli
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ]
